In [1]:
import pandas as pd

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
crime = pd.read_hdf('data/crime.h5')

In [4]:
crime_sort = crime.set_index('REPORTED_DATE').sort_index()

In [5]:
crime_sort.head(5)

OFFENSE_TYPE_ID OFFENSE_CATEGORY_ID  \
REPORTED_DATE                                                             
2012-01-02 00:06:00              aggravated-assault  aggravated-assault   
2012-01-02 00:06:00  violation-of-restraining-order    all-other-crimes   
2012-01-02 00:16:00       traffic-accident-dui-duid    traffic-accident   
2012-01-02 00:47:00                traffic-accident    traffic-accident   
2012-01-02 01:35:00              aggravated-assault  aggravated-assault   

                        GEO_LON    GEO_LAT NEIGHBORHOOD_ID  IS_CRIME  \
REPORTED_DATE                                                          
2012-01-02 00:06:00 -104.816860  39.796717       montbello         1   
2012-01-02 00:06:00 -104.816860  39.796717       montbello         1   
2012-01-02 00:16:00 -104.971851  39.736874   cheesman-park         0   
2012-01-02 00:47:00 -104.951824  39.740168   congress-park         0   
2012-01-02 01:35:00 -104.998131  39.749922   union-station         1   

                     IS_TRAFFIC  
REPORTED_DATE                    
2012-01-02 00:06:00           0  
2012-01-02 00:06:00           0  
2012-01-02 00:16:00           1  
2012-01-02 00:47:00           1  
2012-01-02 01:35:00           0

In [6]:
crime_sort.index.max()   # latest data collected

Timestamp('2017-09-29 06:16:00')

In [7]:
crime_sort = crime_sort[:'2017-8']     # remove september data as it is not collected for total month
crime_sort.index.max()

Timestamp('2017-08-31 23:52:00')

In [8]:
crime_sort.index.min()

Timestamp('2012-01-02 00:06:00')

In [9]:
crimes_per_month = crime_sort.groupby([pd.Grouper(freq='M'), 'OFFENSE_CATEGORY_ID']).size()

In [10]:
crimes_per_month

REPORTED_DATE  OFFENSE_CATEGORY_ID     
2012-01-31     aggravated-assault           113
               all-other-crimes             124
               arson                          5
               auto-theft                   275
               burglary                     343
                                           ... 
2017-08-31     robbery                      108
               sexual-assault                57
               theft-from-motor-vehicle     675
               traffic-accident            2126
               white-collar-crime           138
Length: 1020, dtype: int64

In [11]:
crimes_per_month = crimes_per_month.sort_values().reset_index(name='Total')

In [12]:
crimes_per_month.head(5)

REPORTED_DATE OFFENSE_CATEGORY_ID  Total
0    2012-01-31              murder      0
1    2016-05-31              murder      1
2    2013-01-31               arson      1
3    2016-12-31              murder      1
4    2014-12-31              murder      1

In [13]:
crime_latest_month = crimes_per_month[crimes_per_month['REPORTED_DATE'] == '2017-8-31']

In [14]:
crime_latest_month

REPORTED_DATE           OFFENSE_CATEGORY_ID  Total
80      2017-08-31                        murder      7
81      2017-08-31                         arson      7
189     2017-08-31                sexual-assault     57
313     2017-08-31                       robbery    108
367     2017-08-31            white-collar-crime    138
443     2017-08-31            aggravated-assault    195
561     2017-08-31  other-crimes-against-persons    376
644     2017-08-31                      burglary    432
769     2017-08-31                    auto-theft    599
793     2017-08-31                  drug-alcohol    636
810     2017-08-31      theft-from-motor-vehicle    675
885     2017-08-31                       larceny    877
886     2017-08-31               public-disorder    878
959     2017-08-31              all-other-crimes   1583
1016    2017-08-31              traffic-accident   2126

In [15]:
crime_latest_month['Total_Goal_20_percent'] = crime_latest_month['Total'].mul(.8).astype(int)
crime_latest_month.head()

<ipython-input-15-db2e9b63ea72>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crime_latest_month['Total_Goal_20_percent'] = crime_latest_month['Total'].mul(.8).astype(int)


REPORTED_DATE OFFENSE_CATEGORY_ID  Total  Total_Goal_20_percent
80     2017-08-31              murder      7                      5
81     2017-08-31               arson      7                      5
189    2017-08-31      sexual-assault     57                     45
313    2017-08-31             robbery    108                     86
367    2017-08-31  white-collar-crime    138                    110

In [16]:
pd.merge_asof(crime_latest_month, crimes_per_month, 
              left_on='Total_Goal_20_percent',right_on='Total', 
              by='OFFENSE_CATEGORY_ID', suffixes=('_Current', '_Last'))

REPORTED_DATE_Current           OFFENSE_CATEGORY_ID  Total_Current  \
0             2017-08-31                        murder              7   
1             2017-08-31                         arson              7   
2             2017-08-31                sexual-assault             57   
3             2017-08-31                       robbery            108   
4             2017-08-31            white-collar-crime            138   
5             2017-08-31            aggravated-assault            195   
6             2017-08-31  other-crimes-against-persons            376   
7             2017-08-31                      burglary            432   
8             2017-08-31                    auto-theft            599   
9             2017-08-31                  drug-alcohol            636   
10            2017-08-31      theft-from-motor-vehicle            675   
11            2017-08-31                       larceny            877   
12            2017-08-31               public-disorder            878   
13            2017-08-31              all-other-crimes           1583   
14            2017-08-31              traffic-accident           2126   

    Total_Goal_20_percent REPORTED_DATE_Last  Total_Last  
0                       5         2017-05-31           5  
1                       5         2012-01-31           5  
2                      45         2013-01-31          45  
3                      86         2015-03-31          86  
4                     110         2016-10-31         110  
5                     156         2016-05-31         154  
6                     300         2014-04-30         285  
7                     345         2012-01-31         343  
8                     479         2017-07-31         477  
9                     508         2015-05-31         505  
10                    540         2015-03-31         535  
11                    701         2015-01-31         697  
12                    702         2015-12-31         699  
13                   1266         2016-11-30        1264  
14                   1700         2013-12-31        1697

#### Achieving the same using pandas Period object

In [17]:
pd.Period(year=2012, month=5, day=17, hour=14, minute=20, freq='T')

Period('2012-05-17 14:20', 'T')

In [18]:
ad_period = crime_sort.groupby([lambda x: x.to_period('M'), 'OFFENSE_CATEGORY_ID']).size()

In [19]:
ad_period

         OFFENSE_CATEGORY_ID     
2012-01  aggravated-assault           113
         all-other-crimes             124
         arson                          5
         auto-theft                   275
         burglary                     343
                                     ... 
2017-08  robbery                      108
         sexual-assault                57
         theft-from-motor-vehicle     675
         traffic-accident            2126
         white-collar-crime           138
Length: 1020, dtype: int64

In [20]:
ad_period = ad_period.sort_values().reset_index(name='Total').rename(columns={'level_0':'REPORTED_DATE'})

In [21]:
ad_period

REPORTED_DATE OFFENSE_CATEGORY_ID  Total
0          2012-01              murder      0
1          2016-05              murder      1
2          2013-01               arson      1
3          2016-12              murder      1
4          2014-12              murder      1
...            ...                 ...    ...
1015       2016-09    traffic-accident   2120
1016       2017-08    traffic-accident   2126
1017       2016-10    traffic-accident   2156
1018       2015-10    traffic-accident   2156
1019       2017-05    traffic-accident   2199

[1020 rows x 3 columns]

In [22]:
cols = ['OFFENSE_CATEGORY_ID', 'Total']
crimes_per_month[cols].equals(ad_period[cols])

True

In [23]:
aug_2018 = pd.Period('2017-8', freq='M')

In [24]:
goal_period = ad_period[ad_period['REPORTED_DATE'] == aug_2018].reset_index(drop=True)

In [25]:
goal_period

REPORTED_DATE           OFFENSE_CATEGORY_ID  Total
0        2017-08                        murder      7
1        2017-08                         arson      7
2        2017-08                sexual-assault     57
3        2017-08                       robbery    108
4        2017-08            white-collar-crime    138
5        2017-08            aggravated-assault    195
6        2017-08  other-crimes-against-persons    376
7        2017-08                      burglary    432
8        2017-08                    auto-theft    599
9        2017-08                  drug-alcohol    636
10       2017-08      theft-from-motor-vehicle    675
11       2017-08                       larceny    877
12       2017-08               public-disorder    878
13       2017-08              all-other-crimes   1583
14       2017-08              traffic-accident   2126

In [26]:
goal_period['Total_Goal_20_percent'] = goal_period['Total'].mul(.8).astype(int)

In [27]:
goal_period

REPORTED_DATE           OFFENSE_CATEGORY_ID  Total  Total_Goal_20_percent
0        2017-08                        murder      7                      5
1        2017-08                         arson      7                      5
2        2017-08                sexual-assault     57                     45
3        2017-08                       robbery    108                     86
4        2017-08            white-collar-crime    138                    110
5        2017-08            aggravated-assault    195                    156
6        2017-08  other-crimes-against-persons    376                    300
7        2017-08                      burglary    432                    345
8        2017-08                    auto-theft    599                    479
9        2017-08                  drug-alcohol    636                    508
10       2017-08      theft-from-motor-vehicle    675                    540
11       2017-08                       larceny    877                    701
12       2017-08               public-disorder    878                    702
13       2017-08              all-other-crimes   1583                   1266
14       2017-08              traffic-accident   2126                   1700

In [28]:
pd.merge_asof(goal_period, ad_period, 
              left_on='Total_Goal_20_percent',right_on='Total', 
              by='OFFENSE_CATEGORY_ID', suffixes=('_Current', '_Last'))

REPORTED_DATE_Current           OFFENSE_CATEGORY_ID  Total_Current  \
0                2017-08                        murder              7   
1                2017-08                         arson              7   
2                2017-08                sexual-assault             57   
3                2017-08                       robbery            108   
4                2017-08            white-collar-crime            138   
5                2017-08            aggravated-assault            195   
6                2017-08  other-crimes-against-persons            376   
7                2017-08                      burglary            432   
8                2017-08                    auto-theft            599   
9                2017-08                  drug-alcohol            636   
10               2017-08      theft-from-motor-vehicle            675   
11               2017-08                       larceny            877   
12               2017-08               public-disorder            878   
13               2017-08              all-other-crimes           1583   
14               2017-08              traffic-accident           2126   

    Total_Goal_20_percent REPORTED_DATE_Last  Total_Last  
0                       5            2017-05           5  
1                       5            2012-01           5  
2                      45            2013-01          45  
3                      86            2015-03          86  
4                     110            2016-10         110  
5                     156            2016-05         154  
6                     300            2014-04         285  
7                     345            2012-01         343  
8                     479            2017-07         477  
9                     508            2015-05         505  
10                    540            2015-03         535  
11                    701            2015-01         697  
12                    702            2015-12         699  
13                   1266            2016-11        1264  
14                   1700            2013-12        1697